In [6]:
import random
import numpy as np

In [16]:
def fill_region(array_map, n,  top_left, top_right, bot_left, bot_right):
    # top_left
    while top_left[0] >= 1 and top_left[1] >= 1:
        top_left[0] -= 1
        top_left[1] -= 1
        for i in range(top_left[1], int(n/2-1) + 1):
            array_map[top_left[0]][i] = array_map[int(n/2-1)][int(n/2-1)]
            array_map[i][top_left[0]] = array_map[int(n/2-1)][int(n/2-1)]
    # top_right
    while top_right[0] >= 1 and top_right[1] < n-1:
        top_right[0] -= 1
        top_right[1] += 1
        for i in reversed(range(int(n/2), top_right[1] + 1)):
            array_map[top_right[0]][i] = array_map[int(n/2) - 1][int(n/2)]
        for i in range(top_right[0], int(n/2)):
            array_map[i][top_right[1]] = array_map[int(n/2) - 1][int(n/2)]

    # bot_left
    while bot_left[0] < n-1 and bot_left[1] >= 1:
        bot_left[0] += 1
        bot_left[1] -= 1
        for i in reversed(range(int(n/2), bot_left[0] + 1)):
            array_map[i][bot_left[1]] = array_map[int(n/2)][int(n/2) - 1]
        for i in range(bot_left[1], int(n/2 - 1) + 1):
            array_map[bot_left[0]][i] = array_map[int(n/2)][int(n/2) - 1]

    # bot_right
    while bot_right[0] < n-1 and bot_right[1] < n-1:
        bot_right[0] += 1
        bot_right[1] += 1
        for i in reversed(range(int(n/2), bot_right[0] + 1)):
            array_map[i][bot_right[1]] = array_map[int(n/2)][int(n/2)]
            array_map[bot_right[1]][i] = array_map[int(n/2)][int(n/2)]

    return array_map


def gen_r_left(array_map, n, re_left):
    for i in range(4, 6):
        index_row = random.randint(4, n-4)
        index_col = random.randint(4, n-4)
        size = random.randint(int(n/4), int(n/2))
        #print(f"number {str(i)} with row: {index_row} and col: {index_col} and size: {size}")
        for j in range(index_row, n - 2):
            cnt = 0
            for k in range(index_col, n - 2):
                if cnt == size:
                    break
                if j < n - 1 and k < n - 1:
                    array_map[j][k] = str(re_left[i])
                cnt += 1
    return array_map


def gen_M_and_P_and_T(array_map, n):
    ranPirate = []
    for i in range(1, 7):
        if str(i) in array_map:
            prison = list(zip(*np.where(array_map == str(i))))
            rand_prison = random.randint(0, len(prison) - 1)
            array_map[prison[rand_prison][0]][prison[rand_prison][1]] += "P"
            ranPirate.append((prison[rand_prison][0], prison[rand_prison][1]))

    for i in range(n):
        for j in range(n):
            mountain = random.randint(0, 5)  # 0: yes, >=1: no
            if mountain == 0 and "P" not in array_map[i][j] and array_map[i][j] != "0":
                array_map[i][j] = array_map[i][j] + "M"
    while True:
        row = random.randint(1, n-2)
        col = random.randint(1, n-2)
        if array_map[row][col] != '0' and "P" not in array_map[row][col] and "M" not in array_map[row][col]:
            array_map[row][col] += "T"
            return (array_map, (row, col))


def fix_missing_region(array_map, n, regions):
    arr_tmp = np.array(array_map, dtype=np.int32)
    fix = []
    for i in range(1, regions):
        if str(i) not in array_map:
            fix.append(str(i))
    deleteNum = arr_tmp.argsort()[len(fix):]
    print(fix)
    print("delete:", deleteNum)


def gen_map(array_map, n):
    # 1->6
    regions = 7

    re_left = random.sample(range(1, regions), 6)

    top_left = [int((n/2)-1), int((n/2)-1)]
    top_right = [int((n/2)-1), int((n/2))]
    bot_left = [int((n/2)), int((n/2)-1)]
    bot_right = [int((n/2)), int((n/2))]

    array_map[top_left[0]][top_left[1]] = str(re_left[0])
    array_map[top_right[0]][top_right[1]] = str(re_left[1])
    array_map[bot_left[0]][bot_left[1]] = str(re_left[2])
    array_map[bot_right[0]][bot_right[1]] = str(re_left[3])

    array_map = fill_region(array_map, n, top_left, top_right, bot_left, bot_right)
    array_map = gen_r_left(array_map, n, re_left)

    # random sea
    array_map[0] = "0"
    array_map[n-1] = "0"
    array_map[:, 0] = "0"
    array_map[:, n-1] = "0"
    for i in range(1, n-2):
        sea = random.randint(0, 3)
        # print(sea)

        left_right = random.randint(0, 1)  # 0: left, 1: right
        up_down = random.randint(0, 1)  # 0: up, 1: down

        if left_right == 0:
            for j in range(sea):
                array_map[i][j] = "0"
        else:
            for j in range(n - 1 - sea, n - 1):
                array_map[i][j] = "0"

        if up_down == 0:
            for k in range(sea):
                array_map[k][i] = "0"
        else:
            for h in range(n - 1 - sea, n - 1):
                array_map[h][i] = "0"
    #fix_missing_region(array_map, n, regions)
    array_map = gen_M_and_P_and_T(array_map, n)
    return array_map


class Cell:
    def __init__(self, row, col, dist, preStep):
        self.row = row
        self.col = col
        self.dist = dist
        self.preStep = list(preStep)


def minDistance(array_map, source):

    # To maintain location visit status
    visited = [[False for _ in range(len(array_map[0]))]
               for _ in range(len(array_map))]

    # applying BFS on matrix cells starting from source
    queue = []
    queue.append(source)
    visited[source.row][source.col] = True
    while len(queue) != 0:
        source = queue.pop(0)
        # Destination found;
        if ("T" in array_map[source.row][source.col]):
            return source

        # moving up
        if isValid(source.row - 1, source.col, array_map, visited):
            nextStep = Cell(source.row - 1, source.col,
                            source.dist + 1, source.preStep)
            nextStep.preStep.append(((nextStep.row, nextStep.col)))
            queue.append(nextStep)
            #print("up", source.row, source.col)
            visited[source.row - 1][source.col] = True

        # moving down
        if isValid(source.row + 1, source.col, array_map, visited):
            nextStep = Cell(source.row + 1, source.col,
                            source.dist + 1, source.preStep)
            nextStep.preStep.append((nextStep.row, nextStep.col))
            queue.append(nextStep)
            #print("down", source.row, source.col)
            visited[source.row + 1][source.col] = True

        # moving left
        if isValid(source.row, source.col - 1, array_map, visited):
            nextStep = Cell(source.row, source.col - 1,
                            source.dist + 1, source.preStep)
            nextStep.preStep.append((nextStep.row, nextStep.col))
            queue.append(nextStep)
            #print("left", source.row, source.col)
            visited[source.row][source.col - 1] = True

        # moving right
        if isValid(source.row, source.col + 1, array_map, visited):
            nextStep = Cell(source.row, source.col + 1,
                            source.dist + 1, source.preStep)
            nextStep.preStep.append((nextStep.row, nextStep.col))
            queue.append(nextStep)
            #print("right", source.row, source.col)
            visited[source.row][source.col + 1] = True

    return -1


def isValid(x, y, array_map, visited):
    if ((x >= 0 and y >= 0) and
        (x < len(array_map) and y < len(array_map[0])) and
            (array_map[x][y] != '0') and ("M" not in array_map[x][y]) and (visited[x][y] == False)):
        return True
    return False

In [17]:
def save_map():
    while True:
        n = 16
        array_map = np.chararray([n, n], 3, "utf-8")
        array_map, treasure = gen_map(array_map, n)
   
        with open("MAP_0.txt", "w") as f:
            # - The first line contains 2 numbers to represent the size of the map (𝑾 and 𝑯).
            f.write(" ".join([str(n), str(n)]))
            # - The second line is the turn number that the pirate reveals the prison (𝒓).
            f.write('\n2')
            # - The third line is the turn number that the pirate is free and start running to the
            # treasure. (𝒇).
            f.write('\n4')
            # - The fourth line is the number of regions on the map (including the sea) (𝑹).
            f.write("\n4")
            # - The fifth line contains 2 numbers to represent the treasure position (𝑻𝒙, 𝑻𝒚)
            f.write("\n" + str(treasure[0]) + " " + str(treasure[1]))
            # - The next H lines are the H rows of a map
            for row in array_map:
                f.write("\n")
                cnt = 0
                for elm in row:
                    cnt += 1
                    f.write(elm)
                    if (cnt < n):
                        if (len(elm) < 2):
                            f.write(" ")
                        f.write(";")
        map_available = True
        for i in range(n):
            for j in range(n):
                if (array_map[i][j].endswith("P")):
                    source = Cell(0, 0, 0, [])
                    source.row = i
                    source.col = j
                    source.preStep.append((source.row, source.col))
                    res = minDistance(array_map, source)
                    if (res == -1):
                        map_available = False
        if (map_available):
            break;
        else:
            print("Gen map failed: ")

In [18]:
save_map()